```text
SPDX-FileCopyrightText: 2023 Google LLC
SPDX-License-Identifier: Apache-2.0
```


# Using Google Cloud from Colab

<center>
<table><tr>
<td><a href="https://colab.research.google.com/github/PicardParis/cloud-snippets/blob/main/python/colab/Using Google Cloud from Colab.ipynb">
<img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo" align="center"> Run in Colab
</a></td>
<td><a href="https://github.com/PicardParis/cloud-snippets/blob/main/python/colab/Using Google Cloud from Colab.ipynb">
<img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo" align="center"> View on GitHub
</a></td>
</tr></table>
</center>

Colab now offers a simpler way to use Google Cloud services from a notebook with the `authenticate_user(project_id=...)` method.

To check this out, let's see how to analyze text with machine learning, using the [Natural Language API](https://cloud.google.com/natural-language/docs/).


## 1️⃣ Install the client library

The client library for the Natural Language API is `google-cloud-language`. Make sure a recent version is installed:


In [ ]:
%pip install google-cloud-language>=2.9.1

> If you get a _"You must restart the runtime in order to use newly installed versions"_ warning: Click the `RESTART RUNTIME` button.


## 2️⃣ Define your Project ID

This example assumes you have an operational Google Cloud project:

- If needed, [create a new project](https://console.cloud.google.com/cloud-resource-manager).
- Make sure to [enable billing](https://console.cloud.google.com/billing/linkedaccount) for this project.
- This notebook will not generate any cost (unless you run it thousands of times or with very large text). See the Natural Language API [free monthly thresholds](https://cloud.google.com/natural-language/pricing)).


In [ ]:
# @title {display-mode: "form"}

PROJECT_ID = ""  # @param {type:"string"}

assert PROJECT_ID, "❌ Please enter your project ID"

print(f'✔️ PROJECT_ID: "{PROJECT_ID}"')

## 3️⃣ Authenticate


In [ ]:
from google.colab import auth as google_auth

google_auth.authenticate_user(project_id=PROJECT_ID)

print(f"✔️ Authenticated")

At this stage:

- Your Colab notebook can access your Drive files (not used here).
- You are authenticated for the `gcloud` CLI commands.
- You are also authenticated for the client libraries (**new**). Note that **you generally won't need to create a service account.**
- Your default project ID is set (**new**).


## 4️⃣ Enable the API

The service for the Natural Language API is `language.googleapis.com`. Make sure it's enabled for your project:


In [ ]:
api = "language.googleapis.com"
res = !gcloud services list --enabled --format "value(config.name)"

if api in res:
    print(f"✔️ {api} is enabled")
else:
    print(f"🔓 Enabling {api}…")
    !gcloud services enable $api

## 5️⃣ Analyze sentiments in text

Sentiment analysis inspects the given text and identifies the prevailing emotional opinions within the text, especially to determine expressed sentiments as positive, negative, or neutral. It is performed with the `analyze_sentiment` method, both at the sentence and the document levels.


In [ ]:
# @title Functions and input text {display-mode: "form"}
import pandas as pd
from google.cloud import language
from IPython.display import display


def analyze_text_sentiment(text: str) -> language.AnalyzeSentimentResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.analyze_sentiment(document=document)


def show_table(columns, data, formats=None):
    df = pd.DataFrame(columns=columns, data=data)
    # Customize formatting
    styler = df.style
    if formats:
        styler.format(formats)
    # Left-align string columns
    df = df.convert_dtypes()
    str_cols = list(df.select_dtypes("string").keys())
    styler = styler.set_properties(subset=str_cols, **{"text-align": "left"})
    # Center headers
    styler.set_table_styles([{"selector": "th", "props": [("text-align", "center")]}])
    styler.hide()
    display(styler)


def show_text_sentiment(response: language.AnalyzeSentimentResponse):
    columns = ["score", "sentence"]
    data = [(s.sentiment.score, s.text.content) for s in response.sentences]
    formats = {"score": "{:+.1f}"}
    print("At sentence level:")
    show_table(columns, data, formats)

    sentiment = response.document_sentiment
    columns = ["score", "magnitude", "language"]
    data = [(sentiment.score, sentiment.magnitude, response.language)]
    formats = {"score": "{:+.1f}", "magnitude": "{:.1f}"}
    print("")
    print("At document level:")
    show_table(columns, data, formats)


text = "Python is a very readable language, which makes it easy to understand and maintain code. It's simple, very flexible, easy to learn, and suitable for a wide variety of tasks. One disadvantage is its speed: it's not as fast as some other programming languages."  # @param {type:"string"}
assert text, "❌ Please enter the text to be analyzed"

print(f"✔️ Functions and input text defined")

In [ ]:
# @title Analysis

# Send a request to the API
analyze_sentiment_response = analyze_text_sentiment(text)

# Output
show_text_sentiment(analyze_sentiment_response)

## 🎉 Congratulations

You used a Google Cloud service from your notebook in just a few steps!
